In [1]:
from google.colab import drive
drive.mount("/content/drive")

!pip install tonic --quiet
!pip install snntorch --quiet

import gc
import os
import ast
import csv
import torch
import tonic
import numpy as np 
import pandas as pd
import torch.nn as nn
import snntorch as snn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tonic.transforms as transforms
from snntorch import utils
from snntorch import surrogate 
from torchsummary import summary
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

gc.collect()
torch.cuda.empty_cache()

SRC = "/content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/"

# List of .csv files with event data
csv_files = [SRC + 'GT_HW1_1_DROPPED.csv', SRC + 'GT_HW1_3_DROPPED.csv', SRC + 'GT_HW1_4_DROPPED.csv', SRC + 'GT_HW2_1_DROPPED.csv', 
             SRC + 'GT_HW2_2_DROPPED.csv', SRC + 'GT_HW2_3_DROPPED.csv', SRC + 'GT_HW2_4_DROPPED.csv', SRC + 'GT_HW2_5_DROPPED.csv', 
             SRC + 'GT_HW2_6_DROPPED.csv', SRC + 'GT_LAMBDA_1_DROPPED.csv', SRC + 'GT_LAMBDA_2_DROPPED.csv', SRC + 'GT_LAMBDA_3_DROPPED.csv', 
             SRC + 'GT_LAMBDA_4_DROPPED.csv', SRC + 'GT_LAMBDA_5_DROPPED.csv', SRC + 'GT_LAMBDA_6_DROPPED.csv', SRC + 'GT_LAMBDA_7_DROPPED.csv', 
             SRC + 'GT_LAMBDA_8_DROPPED.csv', SRC + 'GT_BOX1_DROPPED.csv', SRC + 'GT_BOX2_DROPPED.csv', SRC + 'GT_FLOOR_DROPPED.csv']

Mounted at /content/drive
     |████████████████████████████████| 81 kB 7.1 MB/s 
     |████████████████████████████████| 113 kB 84.1 MB/s 
     |████████████████████████████████| 92 kB 597 kB/s 


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Fri Apr  1 18:41:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
df_list = []
for file in csv_files:
  # Drop unecessary columns when reading in the dataframes
  df = pd.read_csv(file, index_col = 0).reset_index(drop = True)
  print("Read: ", file)
  print("Number of data points: ", len(df))
  del df['index']
  df_list.append(df)
concat_df = pd.concat(df_list).reset_index(drop = True)
print("Length of all data points: ", len(concat_df))

Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW1_1_DROPPED.csv
Number of data points:  899
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW1_3_DROPPED.csv
Number of data points:  898
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW1_4_DROPPED.csv
Number of data points:  238
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW2_1_DROPPED.csv
Number of data points:  899
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW2_2_DROPPED.csv
Number of data points:  897
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW2_3_DROPPED.csv
Number of data points:  898
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW2_4_DROPPED.csv
Number of data points:  898
Read:  /content/drive/Shareddrives/Shilpa_Thesis/processed_data/dropped/GT_HW2_5_DROPPED.csv
Number of data points:  898
Read:  /content/drive/Shareddriv

In [4]:
# Shuffle the dataframe rows and reset the index before training/validation/test split
shuffled_df = concat_df.sample(frac = 1).reset_index(drop = True)

# Split the concatenated dataframe into training and test subsets
train_df, validation_df = train_test_split(concat_df, test_size = 0.25)
train_df, test_df = train_test_split(train_df, test_size = 0.20)
print("Training dataset size: ", len(train_df))
print("Validation dataset size: ", len(validation_df))
print("Test dataset size: ", len(test_df))

Training dataset size:  14959
Validation dataset size:  6233
Test dataset size:  3740


In [5]:
class SyntheticRecording(tonic.Dataset):
  """
      Synthetic event camera recordings dataset.
  """
  def __init__(self, df):
    super(SyntheticRecording, self).__init__()
    self.df = df.reset_index(drop = True) # Address index out of order issue
    self.events = self.df['Events'] # Select only last column of dataframe
    self.target = self.df[['Vel_x', 'Vel_y', 'Vel_z']] # Select every column except last column of dataframe
    self.sensor_size = (1920, 1080, 2)
    # Denoise removes isolated, one-off events
    self.frame_transform = transforms.Compose([transforms.Denoise(filter_time = 1000000),
                                               transforms.ToFrame(sensor_size = (1920, 1080, 2), n_event_bins = 5)]) 
    
  """
      Retrieve the index i to get the ith sample from the dataset. Apply the appropriate transformations.
  """
  def __getitem__(self, index):
    list_ = ast.literal_eval(self.events[index]) # Convert string literal to list
    t, x, y, p = [], [], [], []
    for e in list_:
      t.append(e[0] * 1e6) # Convert to microseconds
      x.append(e[1])
      y.append(e[2])
      p.append(e[3])
    structured_events = tonic.io.make_structured_array(x, y, t, p) # Ordering is xytp now
    transformed_frames = self.frame_transform(structured_events)
    vel_xyz = []
    vel_x = self.target.loc[index][0]
    vel_xyz.append(vel_x)
    vel_y = self.target.loc[index][1]
    vel_xyz.append(vel_y)
    vel_z = self.target.loc[index][2]
    vel_xyz.append(vel_z)

    frames_tensor = torch.tensor(transformed_frames)
    vel_tensor = torch.tensor(vel_xyz, dtype = torch.float32)
    return frames_tensor, vel_tensor

  """
      Returns the size of the dataset.
  """
  def __len__(self):
    return len(self.df)

class PadMultiOutputTensor:
  def __init__(self, batch_first: bool = False):
    self.batch_first = batch_first

  def __call__(self, batch):
    samples_output = []
    targets_output = []

    max_length = max([sample.shape[0] for sample, target in batch])
    for sample, target in batch:
      if isinstance(sample, torch.Tensor):
        sample = torch.tensor(sample)
        samples_output.append(
            torch.cat((sample,
                       torch.zeros(max_length - sample.shape[0], *sample.shape[1:]),)))
        targets_output.append(target)
        
    return (torch.stack(samples_output, 0 if self.batch_first else 1),
            torch.stack(targets_output),)

In [6]:
# Create PyTorch dataset objects
train_dataset = SyntheticRecording(train_df) 
val_dataset = SyntheticRecording(validation_df)
test_dataset = SyntheticRecording(test_df)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size = 1,
                          collate_fn = PadMultiOutputTensor())
val_loader = DataLoader(val_dataset, batch_size = 1,
                        collate_fn = PadMultiOutputTensor())
test_loader = DataLoader(test_dataset, batch_size = 1,
                         collate_fn = PadMultiOutputTensor())

In [7]:
class MultiOutputSNN(nn.Module):
  def __init__(self, beta, spike_grad):
    super(MultiOutputSNN, self).__init__()
    self.beta = beta
    self.spike_grad = spike_grad

    # Initialize the layers
    self.conv1 = nn.Conv2d(in_channels = 2, out_channels = 8, kernel_size = 5)
    self.lif1 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.lif2 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.lif3 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 5)
    self.lif4 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.lif5 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.lif6 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(in_features = 16 * 267 * 477, out_features = 3)

  def forward(self, x):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()
    mem3 = self.lif3.init_leaky()
    mem4 = self.lif4.init_leaky()
    mem5 = self.lif5.init_leaky()
    mem6 = self.lif6.init_leaky()

    vel_xyz = None

    for step in range(x.size(0)):
      res = F.max_pool2d(self.conv1(x[step]), (2, 2))
      spk1, mem1 = self.lif1(res, mem1)
      spk2, mem2 = self.lif2(spk1, mem2)
      spk3, mem3 = self.lif3(spk2, mem3)
      res2 = F.max_pool2d(self.conv2(spk3), (2, 2))
      spk4, mem4 = self.lif4(res2, mem4)
      spk5, mem5 = self.lif5(spk4, mem5)
      spk6, mem6 = self.lif4(spk5, mem6)
      flat = self.flat(spk6)
      vel_xyz = self.fc1(flat)
      
    return vel_xyz

class MultiOutputSCNN816(nn.Module):
  def __init__(self, alpha, beta, spike_grad):
    super(MultiOutputSCNN816, self).__init__()
    self.alpha = alpha
    self.beta = beta
    self.spike_grad = spike_grad

    # Initialize the layers
    self.conv1 = nn.Conv2d(in_channels = 2, out_channels = 8, kernel_size = 5)
    self.lif1 = snn.Synaptic(alpha = self.alpha, beta = self.beta, spike_grad = self.spike_grad)
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 5)
    self.lif2 = snn.Synaptic(alpha = self.alpha, beta = self.beta, spike_grad = self.spike_grad)
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(in_features = 16 * 267 * 477, out_features = 3)

  def forward(self, x):
    syn1, mem1 = self.lif1.init_synaptic()
    syn2, mem2 = self.lif2.init_synaptic()

    vel_xyz = None

    for step in range(x.size(0)):
      res = F.max_pool2d(self.conv1(x[step]), (2, 2))
      spk1, syn1, mem1 = self.lif1(res, syn1, mem1)
      res2 = F.max_pool2d(self.conv2(spk1), (2, 2))
      spk2, syn2, mem2 = self.lif2(res2, syn2, mem2)
      flat = self.flat(spk2)
      vel_xyz = self.fc1(flat)
      
    return vel_xyz

class MultiOutputSCNN1232(nn.Module):
  def __init__(self, beta, spike_grad):
    super(MultiOutputSCNN1232, self).__init__()
    self.beta = beta
    self.spike_grad = spike_grad

    # Initialize the layers
    self.conv1 = nn.Conv2d(in_channels = 2, out_channels = 12, kernel_size = 5)
    self.lif1 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 32, kernel_size = 5)
    self.lif2 = snn.Leaky(beta = self.beta, spike_grad = self.spike_grad)
    self.flat = nn.Flatten()
    self.fc1 = nn.Linear(in_features = 32 * 267 * 477, out_features = 3)

  def forward(self, x):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()

    vel_xyz = None

    for step in range(x.size(0)):
      res = F.max_pool2d(self.conv1(x[step]), (2, 2))
      spk1, mem1 = self.lif1(res, mem1)
      res2 = F.max_pool2d(self.conv2(spk1), (2, 2))
      spk2, mem2 = self.lif2(res2, mem2)
      flat = self.flat(spk2)
      vel_xyz = self.fc1(flat)
      
    return vel_xyz

In [9]:
# Neuron and simulation parameters
spike_grad = surrogate.fast_sigmoid(slope = 75)
beta = 0.5
alpha = 0.6
torch.cuda.empty_cache()
net = MultiOutputSCNN816(beta, alpha, spike_grad).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr = 1e-8)
loss_fn = nn.MSELoss()

In [ ]:
def training_loop(net, train_loader, val_loader, optimizer, loss_fn):
  num_epochs = 1
  loss_l1 = nn.L1Loss()

  # Store loss history for future plotting
  loss_history, test_loss_history  = [], []
  loss_history_mae, test_loss_history_mae = [], []
  loss, test_loss = 0, 0
  history = dict()
  history_val = dict()
  counter = 0
  val_counter = 0

  for epoch in range(num_epochs):
    batch = iter(train_loader)
    for data, targets in batch: # Training loop
      optimizer.zero_grad() # Clear gradients for next train

      data = data.cuda()
      targets = targets.cuda()
      net.train() # Forward pass
      prediction = net(data) # Predictions

      loss = torch.sqrt(loss_fn(prediction, targets))
      loss_mae = loss_l1(prediction, targets)
      loss_history.append(loss.item())
      loss_history_mae.append(loss_mae.item())

      # Gradient calculation and weight update
      loss.backward() # Backpropagation, compute gradients
      optimizer.step() # Performs the update (apply gradients)
   
      if counter % 10 == 0: # Print every 10 results
        print(f"Training Iteration {counter}: Training RMSE: {loss.item()}, Training MAE: {loss_mae.item()}")
      counter += 1

    with torch.no_grad(): # Test loop - do not track history for backpropagation
      net.eval() # Test forward pass
      test_batch = iter(val_loader)
      for test_data, test_targets in test_batch:
        test_data = test_data.cuda()
        test_targets = test_targets.cuda()
        test_pred = net(test_data) # Predictions

        test_loss = torch.sqrt(loss_fn(test_pred, test_targets))
        test_loss_mae = loss_l1(test_pred, test_targets)
        test_loss_history.append(test_loss.item())
        test_loss_history_mae.append(test_loss_mae.item())

        if val_counter % 10 == 0:
          print(f"Validation Iteration {val_counter}: Validation RMSE: {test_loss.item()}, Validation MAE: {test_loss_mae.item()}")
        val_counter += 1

  history['Training RMSE'] = loss_history
  history['Training MAE'] = loss_history_mae
  history_val['Validation RMSE'] = test_loss_history
  history_val['Validation MAE'] = test_loss_history_mae
  torch.save(net.state_dict(), SRC + 'results/model2.pt')
  return history, history_val

history, history_val = training_loop(net, train_loader, val_loader, optimizer, loss_fn)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training Iteration 0: Training RMSE: 0.3360332250595093, Training MAE: 0.2609909176826477
Training Iteration 10: Training RMSE: 0.10863696038722992, Training MAE: 0.08863562345504761
Training Iteration 20: Training RMSE: 0.2526199519634247, Training MAE: 0.2097843736410141
Training Iteration 30: Training RMSE: 0.07765820622444153, Training MAE: 0.06221061944961548
Training Iteration 40: Training RMSE: 0.003842197824269533, Training MAE: 0.0036070928908884525
Training Iteration 50: Training RMSE: 0.005278261844068766, Training MAE: 0.004791509360074997
Training Iteration 60: Training RMSE: 1.6755582094192505, Training MAE: 1.504555106163025
Training Iteration 70: Training RMSE: 0.1525135487318039, Training MAE: 0.10391738265752792
Training Iteration 80: Training RMSE: 0.04010287672281265, Training MAE: 0.03580465167760849
Training Iteration 90: Training RMSE: 0.9316888451576233, Training MAE: 0.8725128173828125
Training Iteration 100: Training RMSE: 0.36847591400146484, Training MAE: 0.

In [ ]:
history_df = pd.DataFrame.from_dict(history)
history_val_df = pd.DataFrame.from_dict(history_val)
history_df.to_csv(SRC + 'results/history_model2.csv')
history_val_df.to_csv(SRC + 'results/history_val_model2.csv')